### 1. **IDF (Inverse Document Frequency)** - 단어의 희귀성
- 검색어에 포함된 단어가 전체 문서 집합에서 얼마나 희귀한지 측정
- 정보 가치가 높은 희귀한 단어에 더 큰 가중치 부여

### 2. **TF (Term Frequency)의 정교한 활용** - 단어 빈도의 포화
- 특정 문서에서 단어가 얼마나 자주 등장하는지
- *TF-IDF와 달리* BM25는 이 값을 그대로 사용하지 않고 **"단어 빈도 포화(Term-frequency saturation)"** 개념을 도입

### 3. **문서 길이 정규화 (Document Length Normalization)**
- BM25는 문서의 길이를 고려하여 점수 보정
- 1만 단어로 이루어진 긴 보고서보다는 100단어로 이루어진 짧은 요약문에 나오는 것이  
해당 주제와 더 밀접한 관련이 있을 가능성이 높다는 논리를 반영  
</br>
---
## BM25가 TF-IDF보다 뛰어난 점

- **단어 빈도 포화**: 키워드 반복 어뷰징에 강하고, 단어 빈도의 중요성을 보다 현실적으로 모델링
- **문서 길이 정규화**: 긴 문서가 단지 길다는 이유만으로 부당하게 높은 점수를 받는 것을 방지하여 공정한 비교 가능
- **유연한 파라미터**: $k_1$과 $b$를 통해 데이터셋의 특성에 맞게 알고리즘을 튜닝 가능
</br>
---
### k1과 b
- **$k_1$ (Term-frequency saturation controller)**: 단어 빈도(TF)의 영향력을 조절
    - **$k_1$ 값이 낮으면**: 단어 빈도가 조금만 높아져도 점수가 금방 포화 상태  
    → 단어가 몇 번 나오는지보다 출현 여부 자체가 더 중요
    - **$k_1$ 값이 높으면**: 단어 빈도가 점수에 미치는 영향이 더 커짐  
    → 단어가 많이 나올수록 점수가 계속해서 더 많이 오름
    - **일반적인 값**: 1.2 ~ 2.0

In [5]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.document_loaders import WebBaseLoader
import bs4

In [6]:
loader = WebBaseLoader(
    web_paths= ("https://news.naver.com/section/101",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("sa_text", "sa_item_SECTION_HEADLINE")
        )
    )
)

In [7]:
docs = loader.load()

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50
)

In [9]:
splits = text_splitter.split_documents(docs)

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [12]:
vectorstore = Chroma.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

In [ ]:
retriever = vectorstore.as_retriever(
    search_type='mmr',      # 검색 결과나 추천 목록의 **품질**과 **다양성**을 동시에 최적화하기 위한 알고리즘
    search_kwargs={"k" : 1, "fetch_k" : 4}
)

In [ ]:
bm25_retriever = BM25Retriever.from_documents(splits)
bm25_retriever.k = 2

#### retriever 2개를 엮어 앙상블

In [18]:
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, retriever],
                  weights=[0.2, 0.8])

In [19]:
docs = ensemble_retriever.invoke("오늘의 증시")

In [22]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [23]:
template = """
당신은 AI 언어 모델 조수입니다. 당신의 임무는 주어진 사용자 질문에 대해 벡터 데이터베이스에서 관련 문서를 검색할 수 있도록 다섯 가지 다른 버전을 생성하는 것입니다.
사용자 질문에 대한 여러 관점을 생성함으로써, 거리 기반 유사성 검색의 한계를 극복하는 데 도움을 주는 것이 목표입니다.
각 질문은 새 줄로 구분하여 제공하세요. 원본 질문: {question}
"""

prompt_perspectives = ChatPromptTemplate.from_template(template)

In [24]:
generate_queries = (
    prompt_perspectives
    | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)

In [25]:
from langchain.load import dumps, loads

In [27]:
def reciprocal_rank_fusion(results, k=60, top_n=2):
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(docs)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)
   
    reranked_results = [ (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x : x[1], reverse=True)
    ]
    return reranked_results[:top_n]

In [28]:
chain = generate_queries | ensemble_retriever.map() | reciprocal_rank_fusion

In [29]:
docs = chain.invoke("오늘의 증시")

/tmp/ipykernel_1713/814652417.py:11: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  reranked_results = [ (loads(doc), score)


In [30]:
from langchain_core.runnables import RunnablePassthrough

In [31]:
template = """다음 맥락을 바탕으로 질문에 답변할 것
{context}


질문: {question}
"""

In [32]:
prompt = ChatPromptTemplate.from_template(template)

In [33]:
final_chain = (
    {
        "context" : chain,
        "question" : RunnablePassthrough()
    }
    | prompt | ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)
    | StrOutputParser()


)

In [34]:
rt = final_chain.invoke("오늘의 증시")

In [36]:
print(rt)

오늘의 증시는 전반적으로 하락세를 보이고 있습니다.

- 뉴욕증시: 미국 뉴욕증권거래소의 주요 3대 지수(다우존스 등)가 동반 하락했습니다. 이는 미국 경제의 약 70%를 차지하는 서비스업 업황이 관세 여파로 악화되었다는 소식에 따라 투자심리가 위축된 것이 주요 원인으로 분석됩니다.

- 한국 증시: 미국의 관세 불확실성 재부각과 경제 지표 발표 등의 영향으로 코스피는 전일 대비 15.94포인트(0.50%) 하락한 3182.06으로 장을 시작하며 하락 출발했습니다.

전반적으로 글로벌 증시는 관세 이슈와 경제 지표에 대한 우려로 인해 투자심리가 위축되며 하락세를 보이고 있습니다.


In [40]:
from openai import OpenAI

client = OpenAI()
response = client.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=rt
    )

In [41]:
response.stream_to_file("./output.mp3")

/tmp/ipykernel_1713/2157980915.py:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file("./output.mp3")


---
### PDF OCR로 읽기

In [1]:
from langchain_text_splitters import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf

/home/juyeon/miniconda3/envs/openai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
def extract_pdf_elements(path, fname):
    return partition_pdf(
        filename=path + fname,
        extract_images_in_pdf=True,  # PDF에서 이미지를 추출
        infer_table_structure=True,  # 테이블 구조를 추론
        chunking_strategy="by_title",  # 타이틀을 기준으로 텍스트를 블록으로 분할
        max_characters=4000,  # 최대 4000자로 텍스트 블록을 제한
        new_after_n_chars=3800,  # 3800자 이후에 새로운 블록 생성
        combine_text_under_n_chars=2000,  # 2000자 이하의 텍스트는 결합
        image_output_dir_path=path,  # 이미지가 저장될 경로 설정
        languages=['kor']
        # image_output_dir_path=os.path.join(os.getcwd(),"figures"),
    )

In [9]:
raw_data = extract_pdf_elements("./data/", "face_perspective_rev4.pdf")

In [10]:
def categorize_elements(raw_pdf_elements):
    """
    PDF에서 추출한 요소들을 테이블과 텍스트로 분류합니다.
    raw_pdf_elements: unstructured.documents.elements 리스트
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))  # 테이블 요소를 저장
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))  # 텍스트 요소를 저장
    return texts, tables




texts, tables = categorize_elements(raw_data)

In [11]:
texts

["제 |\n\n30411 0 416 (0『68 105414[6 0 101011772400 200 (0001000741110814011 609106061 ㅁ 109\n\n한 국 정 보 통 신 학 회 논 문지 01. 23, 30. 1: 399~406, 4182. 2019\n\n학습 시간 단 축 과 보안 강 화 를 위한 새로운 얼굴 인식 방식 권 주연 ㆍ 반 태 원 *\n\n스 88006! 『2066 【『600910100170 시 00102 07 『【<004009 1[2010 ㅁ 179 11016 8300 80609[160109 56040\n\n44- 토 600 6\\00' ㆍ 146-\\00 22077\n\n1000686804866 8640606 1260800060 0 1 300 10[060708000 0081466008, (056008580 을 피 800081 001960916, 065600808101, 52828 01768\n\n2427016590, 1060800060[ 0 1 800 10[0008000 0081466018, (560089808 피 040081 0701061915, (0/60080801, 52828 0068\n\n얼굴 인식 기 술 은 특히 00\\10-19 팬 데 믹 이후 비접촉 신원 의 이미지 기반 시 스 템 은 대규모 학습 데 이 터 셋 으 로 인한 유출 위 험 이라는 두 가지 주요 문 제 에 직 면 하 고 있습니다. 터 와 원근 변 환 을 활 용 한 새로운 좌표 기반 학습 방 식 을 제 안 함 니다. 얼굴 이 미 지 에서 추 출 된 에의 멘 드 마크 과 표 를 활 용 함으로써 원본 이 미 지 를 저장할 필 요 가 없어 개 인 정 보 를 보 호 하고 데이터 크 기 를 줄여 학습 속 도 를 향 상 시 렸습니다, 뜻한 원근 번 환 을 적 용 해 정 면 화 된 데 이 터 른 생 성 함으로써 다양한 각 도 와 표 정 에서도 인식 정확 험 결과, 제 안 된 시 스 템 은 기존 방 식 에 비해 학 습 시 간 을 약 68% 단 축 하 면서도 98.5%

In [12]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=2000, chunk_overlap=200
)

In [13]:
joined_texts = " ".join(texts)
text_token = text_splitter.split_text(joined_texts)

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [15]:
prompt_text = """
당신은 표와 텍스트를 요약하여 검색에 활용할 수 있도록 도와주는 도우미입니다.
이 요약본들은 임베딩되어 원본 텍스트나 표 요소를 검색하는데 사용될 것입니다.
주어진 표나 텍스트의 내용을 검색에 최적화된 간결한 요약으로 작성하세요.
요약할 표 또는 텍스트 : {element}
"""

In [17]:
prompt = ChatPromptTemplate.from_template(prompt_text)

In [18]:
model = ChatOpenAI(model_name="chatgpt-4o-latest", temperature=0)

In [19]:
summaries_chain = (
    {
        'element' : lambda x : x
    }
    | prompt | model | StrOutputParser()
)

text_summaries = summaries_chain.batch(texts, {'max_concurrency' : 5})

In [23]:
import pprint

pprint.pprint(text_summaries)

['요약:  \n'
 '본 논문은 얼굴 인식 기술에서 학습 시간 단축과 개인정보 보호를 동시에 달성하기 위한 새로운 좌표 기반 학습 방식을 제안한다. 원근 '
 '변환과 얼굴 랜드마크를 활용하여 원본 이미지를 저장하지 않고도 정면화된 데이터를 생성함으로써 다양한 각도와 표정에서도 높은 인식 정확도를 '
 '유지한다. 제안된 방식은 기존 대비 약 68%의 학습 시간 단축과 98.5% 이상의 인식 정확도를 달성하였다. 이는 비접촉 생체 인식 '
 '시스템의 보안성과 효율성을 향상시키는 효과적인 방법으로 평가된다.\n'
 '\n'
 '키워드: 얼굴 인식, 딥러닝, 좌표 기반 학습, 원근 변환, 학습 시간 단축, 개인정보 보호.',
 '요약:  \n'
 '본 논문은 얼굴 인식 기술의 정확도 향상과 개인정보 보호를 동시에 달성하기 위한 새로운 접근법을 제안한다. 기존 얼굴 인식 기술은 높은 '
 '정확도를 제공하지만, 원본 이미지 저장으로 인한 개인정보 유출 위험이 존재한다. 이를 해결하기 위해 본 연구는 얼굴 이미지에서 추출한 '
 '68개 랜드마크 좌표를 입력으로 사용하여 원본 이미지 저장 없이 인식이 가능하도록 하였다. 또한, 원근 변환을 적용해 다양한 얼굴 각도와 '
 '표정 변화에 강인한 인식 성능을 확보하였다. 제안된 방법은 기존 방식 대비 약 99%의 인식 정확도를 유지하면서 평균 학습 시간을 약 '
 '68% 단축시켰다. 이 방식은 비접촉 생체 인식 기술의 보안성과 효율성을 동시에 향상시킬 수 있는 대안으로 제시된다.',
 '심화 신경망 기반 얼굴 인식 시스템은 입력 이미지에서 얼굴을 감지하고, 전처리(크기 조정, 정렬, 조명 보정 등)를 거쳐 특징을 추출한 '
 '후 신원을 확인하는 일련의 과정을 포함한다. 얼굴 특징은 주로 68개 랜드마크 좌표를 기반으로 추출되며, 이는 신경망 입력 데이터 크기를 '
 '줄이고 개인 정보 보호에 유리하다. 특징 추출에는 CNN 기반 모델이 사용되며, 완전 연결층(512→256→128→64)을 통해 최종 '
 '64차원